In [2]:
from collections import OrderedDict
import numpy as np
import json
import pandas as pd
from os.path import join

This notebook shows, how to evaluate the MultiLayer Perceptron (MLP) model exported from Julia Flux. See `1.0-hd-export_model_parameters.ipynb`.

In [25]:
model_dir = "../exp/full/8/group_data_size/1024_512_256"

Import all model parameters (weights and biases).

In [20]:
parameters = np.load(join(model_dir, "model.npz"))

Let's have a look at the parameter shapes. These should be transposed versions of those from `1.0-hd-export_model_parameters.ipynb`.

In [24]:
nlayers = len(parameters) // 2
for i in range(1, nlayers):
    W = "W" + str(i)
    b = "b" + str(i)
    print("layer {} weights shape: {}".format(i, parameters[W].shape))
    print("layer {} biases shape: {}".format(i, parameters[b].shape))

layer 1 weights shape: (51, 1024)
layer 1 biases shape: (1, 1024)
layer 2 weights shape: (1024, 512)
layer 2 biases shape: (1, 512)
layer 3 weights shape: (512, 256)
layer 3 biases shape: (1, 256)


The following method evaluates the MLP.

In [43]:
def evaluate(X, parameters, mus, sigmas, final_activation=True):
    nlayers = len(parameters) // 2
    Y = (X-mus)/sigmas # standardize inputs
    print(Y)
    for i in range(1, nlayers+1):
        W = parameters["W" + str(i)]
        b = parameters["b" + str(i)]
        Y = np.dot(Y, W) + b
        if i < nlayers:
            Y = np.maximum(Y, 0)# ReLU for all hidden layers
        elif final_activation:
            Y = 1/(1+np.exp(-Y)) # sigmoid for the output layer so we get expected probability of ride feasibility (values in (0,1])
    return Y   

Now let us see if it gives the same output as the original Julia version.

In [44]:
X = np.ones((1, 51))
mus = np.zeros((1, 51))
sigmas = np.ones((1, 51))
evaluate(X, parameters, mus, sigmas, final_activation=False)

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1.]]


array([[9.70097435]])

The network was trained with standardized inputs so we need to load the standardization coefficients.

In [42]:
model_stds = np.load(join(model_dir, "model_standardization.npz"))
mus = model_stds["mus"].reshape(1, -1)
sigmas = model_stds["sigmas"].reshape(1, -1)

In practise the network is the network will be evaluated as
`evaluate(X, parameters, mus, sigmas, final_activation=True)` where the output is predicted probability of feasible ride.